In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
stations_dir = "/stations_data/"
gcm_pr_dir = "/GCM_PR_Timeseries"
gcm_tm_dir = "/GCM_TM_Timeseries"
stations_file = "/stations_data/stations_coordinates.csv"

stations_df = pd.read_csv(stations_file)

obs_pr_data = []
gcm_pr_data = []
obs_tm_data = []
gcm_tm_data = []

In [ ]:
for index, row in stations_df.iterrows():
    station_name = row["Station"]

    obs_file = os.path.join(stations_dir, f"{station_name}_monthly.csv")
    gcm_pr_file = os.path.join(gcm_pr_dir, f"{station_name}_GCM_PR.csv")
    gcm_tm_file = os.path.join(gcm_tm_dir, f"{station_name}_GCM_PR.csv")

    if os.path.exists(obs_file) and os.path.exists(gcm_pr_file) and os.path.exists(gcm_tm_file):
        obs_data = pd.read_csv(obs_file, usecols=["Precip", "AvgTemp"]).dropna()
        gcm_pr = pd.read_csv(gcm_pr_file, usecols=["Precipitation"]).dropna().values.flatten()
        gcm_tm = pd.read_csv(gcm_tm_file, usecols=["Precipitation"]).dropna().values.flatten()

        min_length = min(len(obs_data), len(gcm_pr), len(gcm_tm))
        obs_data = obs_data.iloc[:min_length]
        gcm_pr = gcm_pr[:min_length]
        gcm_tm = gcm_tm[:min_length]

        obs_pr_data.extend(obs_data["Precip"].values)
        gcm_pr_data.extend(gcm_pr)
        obs_tm_data.extend(obs_data["AvgTemp"].values)
        gcm_tm_data.extend(gcm_tm)

In [ ]:
obs_pr_data = np.array(obs_pr_data)
gcm_pr_data = np.array(gcm_pr_data)
obs_tm_data = np.array(obs_tm_data)
gcm_tm_data = np.array(gcm_tm_data)

mean_obs_pr = np.mean(obs_pr_data)
mean_gcm_pr = np.mean(gcm_pr_data)
mean_obs_tm = np.mean(obs_tm_data)
mean_gcm_tm = np.mean(gcm_tm_data)

pr_5_obs, pr_95_obs = np.percentile(obs_pr_data, [5, 95])
pr_5_gcm, pr_95_gcm = np.percentile(gcm_pr_data, [5, 95])
tm_5_obs, tm_95_obs = np.percentile(obs_tm_data, [5, 95])
tm_5_gcm, tm_95_gcm = np.percentile(gcm_tm_data, [5, 95])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 7))
sns.kdeplot(obs_pr_data, label="Observed Data", color="skyblue", linewidth=1.5, shade=True, alpha=0.6, ax=axes[0])
sns.kdeplot(gcm_pr_data, label="GCM Data", color="salmon", linewidth=1.5, linestyle="--", shade=True, alpha=0.5, ax=axes[0])
axes[0].axvline(mean_obs_pr, color="blue", linestyle="--", linewidth=2)
axes[0].axvline(mean_gcm_pr, color="red", linestyle="--", linewidth=2)
axes[0].axvline(pr_5_obs, color="blue", linestyle=":", linewidth=2, alpha=0.4)
axes[0].axvline(pr_95_obs, color="blue", linestyle=":", linewidth=2, alpha=0.9)
axes[0].axvline(pr_5_gcm, color="red", linestyle=":", linewidth=2, alpha=0.4)
axes[0].axvline(pr_95_gcm, color="red", linestyle=":", linewidth=2, alpha=0.9)
axes[0].set_xlabel("Precipitation (mm/month)", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Density", fontsize=14, fontweight="bold")
axes[0].set_title("Precipitation PDF: Observed vs. GCM", fontsize=16, fontweight="bold")
axes[0].grid(True, linestyle="--", linewidth=0.7, alpha=0.5)

In [ ]:
sns.kdeplot(obs_tm_data, label="Observed Data", color="skyblue", linewidth=1.5, shade=True, alpha=0.6, ax=axes[1])
sns.kdeplot(gcm_tm_data, label="CMIP6 Data", color="salmon", linewidth=1.5, linestyle="--", shade=True, alpha=0.5, ax=axes[1])
axes[1].axvline(mean_obs_tm, color="blue", linestyle="--", linewidth=2)
axes[1].axvline(mean_gcm_tm, color="red", linestyle="--", linewidth=2)
axes[1].axvline(tm_5_obs, color="blue", linestyle=":", linewidth=2, alpha=0.4)
axes[1].axvline(tm_95_obs, color="blue", linestyle=":", linewidth=2, alpha=0.9)
axes[1].axvline(tm_5_gcm, color="red", linestyle=":", linewidth=2, alpha=0.4)
axes[1].axvline(tm_95_gcm, color="red", linestyle=":", linewidth=2, alpha=0.9)
axes[1].set_xlabel("Temperature (°C)", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Density", fontsize=14, fontweight="bold")
axes[1].set_title("Temperature PDF: Observed vs. GCM", fontsize=16, fontweight="bold")
axes[1].grid(True, linestyle="--", linewidth=0.7, alpha=0.5)

In [ ]:
fig.legend(["Observed Data", "CMIP6 Data", "Mean Observed", "Mean CMIP6", "5% Observed", "95% Observed", "5% CMIP6", "95% CMIP6"], loc='lower center', ncol=4, fontsize=12)
fig.subplots_adjust(bottom=0.17)

output_path = "/Users/hosseinsalehi/Downloads/Historical/NetCDF/precip_temp_pdf_comparison.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()